In [ ]:
import random
import pandas as pd
import numpy as np
import os

# 사이킷런에서 랜덤포레스트 모델 불러오기
from sklearn.ensemble import RandomForestRegressor

# 경고 메세지 무시하기
import warnings
warnings.filterwarnings(action='ignore')

!pip install koreanize_matplotlib

import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
import koreanize_matplotlib

if 'google.colab' in str(get_ipython()):
    ! pip install git+https://github.com/ourownstory/neural_prophet.git # 코랩 구동 시 패키지 설치
    # pip install neuralprophet
    # 간단하고 빠르게 설치할 수 있지만, 최신 버전에서의 버그가 발생할 수 있음

#패키지 임포트
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")

#구글 드라이브 다운로드 패키지
from google_drive_downloader import GoogleDriveDownloader as gdd

#데이터 다운로드
gdd.download_file_from_google_drive(file_id='1JRslcolP0XJDQKfXrYl82eyDTCETuzjl',
                                    dest_path='data/energy.zip',
                                    unzip=True)

train = pd.read_csv('train.csv',encoding='cp949')
test = pd.read_csv('test.csv',encoding='cp949')

# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

def make_nan(test):
    
    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')
    
    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    
    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)
    
    return test

test=make_nan(test)

test['power']=0

pip install neuralprophet

pred=[]
for i in range(1,61):
  train_d = pd.DataFrame({"ds": train[train['num']==i]["date_time"], "y": train[train['num']==i]["power"]})
  tes_dt = pd.DataFrame({"ds": test[test['num']==i]["date_time"], "y": test[test['num']==i]["power"]})
  #모델 설정
  m = NeuralProphet(
      learning_rate=0.1, 
  )
  #모델 학습
  metrics = m.fit(train_d, freq="H")
  test_pred = m.predict(tes_dt)
  pred.append(test_pred)

# 'yhat1' 열을 추출할 리스트
yhat1_list = []

# pred 리스트에 있는 각 데이터프레임에서 'yhat1' 열을 추출
for df in pred:
    yhat1_list.append(df['yhat1'])

pd.DataFrame(np.array(yhat1_list).flatten())

submission=pd.read_csv('sample_submission.csv',encoding='cp949')

submission['answer']=pd.DataFrame(np.array(yhat1_list).flatten())

submission

submission.to_csv('submission_prophet.csv',index=False)

In [1]:
1+2

3